# Fault Diagnosis Toolbox in Python
## - Low level structural analysis function calls

Erik Frisk<br>
<frisk@isy.liu.se><br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

In [3]:
%matplotlib notebook
new_paths = ['../Misc/', 'build/lib.macosx-10.6-x86_64-3.5/']

import matplotlib
import matplotlib.pyplot as plt
import sys
import numpy as np
import scipy.sparse as sp
[sys.path.append(d) for d in new_paths if not d in sys.path];
from misc import *
import structuralanalysis3 as sa
import csv

In [8]:
def ReadMatrixCSV(fileName):
    with open(fileName, 'rt') as csvfile:
        matReader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        size = matReader.__next__() # Assume first line contains size information
        X=np.zeros((int(size[0]),int(size[1])),dtype='float64');
        for edge in matReader:
            X[int(edge[0]),int(edge[1])] = float(edge[2])
        return sp.csc_matrix(X.astype(np.int64))


## Low level function calls

In [11]:
X=ReadMatrixCSV('Xengine.txt')
#p,q,r,s,_,_=sa.dmperm_internal(dmperm.CSCDict(X))
#print p

In [ ]:
plt.figure(10)
plt.subplot(1,2,1)
plt.spy(X, marker="o", markersize=4);
plt.subplot(1,2,2)
plt.spy(X[p,:][:,q], marker="o", markersize=4);
plt.show()

In [ ]:
plt.figure(11)
fdt.PlotDM(X,verbose=False,eqclass=False)
plt.show()

## Try some basic dmperm functionality  and time performance

In [ ]:
reload(dmperm);

In [ ]:
row=dmperm.Mplus(X);
print row

In [ ]:
dm=dmperm.GetDMParts(X)
dmperm.sprank(X)

Evaluate timing performance by doing 1000 dmperm 

Matlab comparisson (my iMac):<br>
tic; for k=1:1000; GetDMParts(X); end; toc<br>
Elapsed time is 0.108608 seconds.


In [ ]:
import timeit
timeit.timeit('dmperm.GetDMParts(X)', setup="from __main__ import dmperm, X",number=1000)

In [ ]:
print dmperm.IsPSO(X)
print dmperm.IsPSO(X[dm.Mp.row,:][:,dm.Mp.col])

In [ ]:
Xp=X[dm.Mp.row,:][:,dm.Mp.col]
pd=dmperm.PSODecomposition(Xp)
print len(pd['eqclass'])
print len(pd['X0'])

## Try some MSO calculations

In [ ]:
msos=sa.findmso_internal(dmperm.CSCDict(X));
print len(msos)
print len(msos[0])
print msos[0]

In [ ]:
import timeit
timeit.timeit('sa.findmso_internal(dmperm.CSCDict(X))', setup="from __main__ import sa, dmperm, X",number=100)

In [ ]:
mso_n = map(lambda m:len(m), msos)
plt.figure(20)
plt.hist(mso_n,bins=10,normed=False)
plt.title('Histogram - mso sizes')
plt.xlabel('MSO size')
plt.ylabel('Number of MSO sets')
BoxOff()
plt.show()

## Matchings

In [ ]:
X0 = np.array([
     [1, 1, 0, 1, 0, 1, 0, 0],
     [1, 1, 0, 1, 0, 0, 1, 0],
     [0, 0, 1, 1, 0, 0, 0, 1],
     [0, 0, 2, 3, 0, 1, 0, 1],
     [0, 0, 0, 0, 3, 2, 1, 0],
     [0, 0, 0, 0, 0, 1, 0, 1],
     [0, 0, 0, 0, 0, 0, 2, 3],
     [0, 0, 0, 0, 0, 0, 0, 1]])
p = np.random.permutation(8)
q = np.random.permutation(8)
#p = np.arange(0,8)
#q = np.arange(0,8)
X = X0.copy()[p,:][:,q]
G = (np.arange(0,X0.shape[0], dtype=np.int64), np.arange(0,X0.shape[1], dtype=np.int64), X)
Gamma = dmperm.MixedCausalityMatching(G)
print Gamma